In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::142401413602:role/service-role/AmazonSageMaker-ExecutionRole-20220518T150261
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 6c493b44-6982-4de3-aade-0a70d10e60f0
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session 6c493b44-6982-4de3-aade-0a70d10e60f0 to get into ready status...
Session 6c493b44-6982-4de3-aade-0a70d10e60f0 has been created.



In [2]:
spark = SparkSession.builder.appName("example").getOrCreate()

In [11]:
df = spark.sql(
"""

select 
 cd_cpf
, id_pessoa
, ds_classif_behavior
, ds_seg_consumo
, ds_perfil_research
, ds_persona
from customer_analytics_zone.ca_analitico_basao_crm

"""
)


In [12]:
df.show(5)

+-----------+--------------------+-------------------+--------------------+--------------------+--------------------+
|     cd_cpf|           id_pessoa|ds_classif_behavior|      ds_seg_consumo|  ds_perfil_research|          ds_persona|
+-----------+--------------------+-------------------+--------------------+--------------------+--------------------+
|08649985548|05e078a50ee1edea9...|                 14|                null|       Multiplicador|26 - Homem, Até 2...|
|09633852919|f2459ceaeea8ee564...|                  9|LOJAS DE DEPARTAM...|Multiplicador / A...|17 - Mulher, 25 a...|
|70016377699|22de21058a44a5d7f...|               null|                null|                null|17 - Mulher, 25 a...|
|27744355873|0c5eb96017dd4ff4c...|                 17|                null|       Multiplicador|22 - Mulher, 41 a...|
|07321571459|7e7a4cf9a0a96d3ea...|                 16|                null|BP - Multiplicado...|0 - Mulher, Casad...|
+-----------+--------------------+-------------------+--

In [5]:
df.head(1)

[Row(id_pessoa='05e078a50ee1edea9f634b74451f27e4a8be0b336e05d32466830849dc051322', ds_classif_behavior='14', ds_seg_consumo=None, ds_perfil_research='Multiplicador', ds_persona='26 - Homem, Até 24 anos, Nordeste')]


In [6]:
df.count()

5504274


In [7]:
df3 = spark.sql(
"""

                            SELECT cpf,
                                          COUNT(DISTINCT SUBSTRING(CAST(dt_event AS string), 1, 10)) AS dias_uso_app
								     FROM customer_curated_zone.ca_analitico_geoloc
                                           GROUP BY 1

"""
)

In [8]:
df3.count()

2779510


In [14]:
from pyspark.sql.functions import when, col, countDistinct

clusterization1 = df.select("cd_cpf","ds_perfil_research") \
                        .withColumnRenamed("cd_cpf", "cpf") \
                        .join(df3, "cpf", "left_outer") \
                        .withColumn("dias_uso_app2", when(col("dias_uso_app") > 30, 30).otherwise(col("dias_uso_app")))

In [15]:
clusterization1.show()

+-----------+--------------------+------------+-------------+
|        cpf|  ds_perfil_research|dias_uso_app|dias_uso_app2|
+-----------+--------------------+------------+-------------+
|00000048399|       Multiplicador|          73|           30|
|00000074128|                null|           3|            3|
|00000083542|Multiplicador / A...|          20|           20|
|00000122114|       Multiplicador|        null|         null|
|00000154580|       Multiplicador|          15|           15|
|00000189103|          Acumulador|          68|           30|
|00000420565|BP - Multiplicado...|         258|           30|
|00000651290|                null|        null|         null|
|00000724513|                null|          21|           21|
|00001136208|Multiplicador / B...|          12|           12|
|00001142356|       Multiplicador|          58|           30|
|00001162543|Multiplicador / A...|          86|           30|
|00001327305|Multiplicador / A...|        null|         null|
|0000140

In [16]:
agg1 = df.groupby("ds_seg_consumo") \
.count()

agg1.show(50)


+--------------------+------+
|      ds_seg_consumo| count|
+--------------------+------+
|           SAUDE - 4|  8546|
|        FINANCAS - 2|  4796|
|         TELECOM - 5|  1569|
|  ENTRETENIMENTO - 4|  2406|
|        SERVICOS - 5| 88479|
|      MOBILIDADE - 6| 57128|
|        SERVICOS - 1| 38322|
|      MOBILIDADE - 1| 10338|
|             PET - 2|   744|
|             PET - 6|   977|
|        FINANCAS - 3| 40220|
|        EDUCACAO - 2|  3379|
|             PET - 4|  2972|
|  ENTRETENIMENTO - 2|  5468|
|        DELIVERY - 3|  2522|
|          SEGURO - 3|  1810|
|        SERVICOS - 3| 45292|
|        FINANCAS - 4| 40339|
|        EDUCACAO - 1|  1845|
|      E-COMMERCE - 5| 72617|
|      E-COMMERCE - 1| 16274|
|        DELIVERY - 6|  2345|
|SUPRIMENTOS PARA ...|  6356|
|        EDUCACAO - 4|   773|
|          VIAGEM - 4|   777|
|LOJAS DE DEPARTAM...| 25542|
|        EDUCACAO - 5|  8218|
|      MOBILIDADE - 3|  7247|
|          VIAGEM - 1|   779|
|             PET - 1|   194|
|LOJAS DE 

In [17]:
clusterization1.filter(col("dias_uso_app2") >= 1) \
.groupby("dias_uso_app2", "ds_perfil_research") \
.count() \
.show()

+-------------+--------------------+-----+
|dias_uso_app2|  ds_perfil_research|count|
+-------------+--------------------+-----+
|            5|Multiplicador / A...|12976|
|            5|BP - Multiplicado...| 4597|
|           10|Multiplicador / A...| 8979|
|           15|Multiplicador / A...| 7016|
|            6|Multiplicador / A...|11727|
|           20|          Acumulador| 3112|
|           11|Multiplicador / B...| 6254|
|            9|          Acumulador| 5471|
|           11|  Barrado no Crédito| 4356|
|           19|Multiplicador / A...| 5881|
|           14|BP - Multiplicado...| 2831|
|            9|  Barrado no Crédito| 4784|
|           25|BP - Multiplicado...| 2444|
|            4|BP - Multiplicado...| 4903|
|           24|BP - Multiplicado...| 2036|
|           15|BP - Multiplicado...| 2733|
|           14|  Barrado no Crédito| 3488|
|           28|  Barrado no Crédito| 1744|
|           30|                null|11757|
|           29|                null|  938|
+----------

In [19]:
clusterization1.filter(col("dias_uso_app2") >= 1) \
.groupby("dias_uso_app2", "ds_perfil_research") \
.count() \
.groupby("dias_uso_app2") \
.pivot("ds_perfil_research").sum("count") \
.orderBy(col("dias_uso_app2").asc()).show()

+-------------+-----+----------+-------------------------------+---------------------------------------+------------------+-------------+--------------------------+----------------------------------+-----+
|dias_uso_app2| null|Acumulador|BP - Multiplicador / Acumulador|BP - Multiplicador / Barrado no Crédito|Barrado no Crédito|Multiplicador|Multiplicador / Acumulador|Multiplicador / Barrado no Crédito|   NI|
+-------------+-----+----------+-------------------------------+---------------------------------------+------------------+-------------+--------------------------+----------------------------------+-----+
|            1|76017|     20875|                          15209|                                  10011|             13774|        48030|                     29390|                             19093|14344|
|            2|55409|     13485|                          10029|                                   6753|              9375|        33594|                     20323|            

In [20]:
#df.groupBy("department") \
#    .agg(sum("salary").alias("sum_salary"), \
#         avg("salary").alias("avg_salary"), \
#         sum("bonus").alias("sum_bonus"), \
#         max("bonus").alias("max_bonus") ) \
#    .show(truncate=False)


TypeError: unsupported operand type(s) for +: 'int' and 'str'


In [25]:
df_test = df.drop('cd_cpf').sample(0.001)

In [26]:
df_test.count()

5444


In [27]:
df_test.write.partitionBy("id_pessoa").json("s3a://data-workspace-will-prod/flavia-costa/teste_json_basico")

In [28]:
df_test.show()

+--------------------+-------------------+--------------------+--------------------+--------------------+
|           id_pessoa|ds_classif_behavior|      ds_seg_consumo|  ds_perfil_research|          ds_persona|
+--------------------+-------------------+--------------------+--------------------+--------------------+
|41d3b4434fee54617...|                 14|                null|                null|18 - Homem, 25 a ...|
|448608d1a9b4884fc...|                 16|    SUPERMERCADO - 6|Multiplicador / A...|25 - Homem, 41 an...|
|8e560315acb7a93af...|                 17|                null|Multiplicador / B...|10 - Mulher, Solt...|
|5c62ab889e44a299c...|                 18|                null|BP - Multiplicado...|31 - Mulher, 41 a...|
|d2ff2a0c324edafab...|                 16|                null|                  NI|15 - Mulher, 25 a...|
|f2a436f6cb983e88f...|               null|                null|Multiplicador / B...|23 - Homem, 41 an...|
|30d85e5d326f58d8e...|               null|    

In [49]:
from pyspark.sql.functions import posexplode
from pyspark.sql.functions import posexplode_outer

In [67]:
from pyspark.sql.functions import expr

unpivotExpr = """
stack(4, 'ds_classif_behavior', ds_classif_behavior
, 'ds_seg_consumo', ds_seg_consumo
, 'ds_perfil_research', ds_perfil_research
, 'ds_persona', ds_persona) 
as (Modelos,Score)
"""

df_formato = df_test.select("id_pessoa", expr(unpivotExpr)) 
df_formato.show(truncate=False)
df_formato.show()

+----------------------------------------------------------------+-------------------+--------------------------------------------------------------+
|id_pessoa                                                       |Modelos            |Score                                                         |
+----------------------------------------------------------------+-------------------+--------------------------------------------------------------+
|41d3b4434fee54617c9f71c5ebe4faa0d8a50fb06bdc668b6e6632d0dad1fc89|ds_classif_behavior|14                                                            |
|41d3b4434fee54617c9f71c5ebe4faa0d8a50fb06bdc668b6e6632d0dad1fc89|ds_seg_consumo     |null                                                          |
|41d3b4434fee54617c9f71c5ebe4faa0d8a50fb06bdc668b6e6632d0dad1fc89|ds_perfil_research |null                                                          |
|41d3b4434fee54617c9f71c5ebe4faa0d8a50fb06bdc668b6e6632d0dad1fc89|ds_persona         |18 - Homem, 25

In [75]:
df_formato = df_formato.withColumnRenamed('Modelos', 'id').withColumnRenamed('Score', 'score')\
.withColumn('description', when(col("id") == "ds_classif_behavior", "Behavior").otherwise(\
when(col("id") == "ds_seg_consumo", "Segm.MCC").otherwise(\
when(col("id") == "ds_perfil_research", "Perfis").otherwise(\
when(col("id") == "ds_persona", "Persona").otherwise("")))))

In [76]:
df_formato.show()

+--------------------+-------------------+--------------------+-----------+
|           id_pessoa|                 id|               score|description|
+--------------------+-------------------+--------------------+-----------+
|41d3b4434fee54617...|ds_classif_behavior|                  14|   Behavior|
|41d3b4434fee54617...|     ds_seg_consumo|                null|   Segm.MCC|
|41d3b4434fee54617...| ds_perfil_research|                null|     Perfis|
|41d3b4434fee54617...|         ds_persona|18 - Homem, 25 a ...|    Persona|
|448608d1a9b4884fc...|ds_classif_behavior|                  16|   Behavior|
|448608d1a9b4884fc...|     ds_seg_consumo|    SUPERMERCADO - 6|   Segm.MCC|
|448608d1a9b4884fc...| ds_perfil_research|Multiplicador / A...|     Perfis|
|448608d1a9b4884fc...|         ds_persona|25 - Homem, 41 an...|    Persona|
|8e560315acb7a93af...|ds_classif_behavior|                  17|   Behavior|
|8e560315acb7a93af...|     ds_seg_consumo|                null|   Segm.MCC|
|8e560315acb

In [77]:
import pandas as pd

In [79]:
pandas_df = df_formato.toPandas()

In [80]:
pandas_df.head()

                                           id_pessoa  ... description
0  41d3b4434fee54617c9f71c5ebe4faa0d8a50fb06bdc66...  ...    Behavior
1  41d3b4434fee54617c9f71c5ebe4faa0d8a50fb06bdc66...  ...    Segm.MCC
2  41d3b4434fee54617c9f71c5ebe4faa0d8a50fb06bdc66...  ...      Perfis
3  41d3b4434fee54617c9f71c5ebe4faa0d8a50fb06bdc66...  ...     Persona
4  448608d1a9b4884fc969a7ebb8700f6e1730a85cb7ad3c...  ...    Behavior

[5 rows x 4 columns]


In [ ]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType

@pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)
def g(df):
    result = df.groupby(['id_pessoa'].apply(lambda x: x[['id','description','score']].to_dict('records')).reset_index()
    return result

In [ ]:
j = (df.groupby(['id_pessoa'])\
.apply(g)\
.rename(columns={0:'Models'})\
.to_json(orient='records'))

In [82]:
#j = (df.groupby(['id_pessoa'])\
#.apply(lambda x: x[['id','description','score']].to_dict('records'))\
#.reset_index()\
#.rename(columns={0:'Models'})\
#.to_json(orient='records'))

ValueError: Invalid udf: the udf argument must be a pandas_udf of type GROUPED_MAP.
Stopping session: 6c493b44-6982-4de3-aade-0a70d10e60f0


In [ ]:
print(json.dumps(json.loads(j), indent=2, sort_keys=False)) # ou true para experienceid no final

In [ ]:
df = spark.read.json("./ex.json")
df = df.withColumn("Year", df["year"])
df = df.withColumn("Fruit", df["fruit"])
df.write.partitionBy("Year", "Fruit").json("result")